In [33]:
import pandas as pd
import glob
import os

In [2]:
def remove_duplicates(lst):
    return list(dict.fromkeys(lst).keys())

In [66]:
def cats_to_list(string):
    cat_list_str = string.split(' ')
    return [int(x) for x in cat_list_str]
    

In [6]:
def convert_cats(lst):
    if len(lst) > 1:
        lst = ' '.join([x for x in lst])
    else:
        lst = '[' + str(lst[0]) + ']'
    return lst

In [71]:
def select_top(string):
    string = str(string)
    if '[' not in string:
        top = [int(x) for x in string.split(' ')][0]
        return f'[{top}]'
    else:
        return string

In [68]:
def generate_submission_file(df, output):
    df[['id', 'categories', 'osd']].to_csv(output, index=False)

In [75]:
def remap(cat_lst, mapper):
    out = []
    for i in cat_lst:
        out.append(mapper.get(i))
    return out


def generate_submission_df(input_path = 'runs/detect/predict/labels/'):
    out = {}
    filelist = glob.glob(input_path + '*.txt')

    cat_df = pd.read_json('category_key.json')
    shallow = cat_df[cat_df.shallow_species == True]['index'].to_list()
    mapper = cat_df[['id', 'index']].to_dict()['id']

    for i, file in enumerate(filelist):
        with open(file, 'r') as f:
            conf = []
            cats = []
            # osd = 0.5  # set base case osd, this will hod if there are no shallow or empty list
            for line in f.readlines():
                category = int(line.split(' ')[0])
                if category in shallow: # chech shallow and set osd to 0 if in shallow
                    osd = 0
                if category not in cats: # dedup
                    cats.append(category)
                    conf.append(float(line.split(' ')[-1]))
            
            cats = remap(cats, mapper)

            if len(cats) > 1:
                cats = ' '.join([str(x) for x in cats])
            if len(cats) == 0:
                cats = '[52]'
                osd = 1


        out[i] = {'id': os.path.basename(file)[:-4],
                  'categories': cats,
                  'osd': osd,
                  'conf': conf
                  }

    df = pd.DataFrame.from_dict(out, orient='index')
    return df


In [76]:
df = generate_submission_df('runs/detect/predict2/labels/')
df

,id,categories,osd,conf
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[52],1,[]
1,49254842-f712-4875-ad09-57ebf8508bd1,219 52 218 242,1,"[0.68369, 0.591417, 0.535339, 0.298256]"
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[52],1,[]
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,[160],0,[0.801862]
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,[51],0,[0.561465]
...,...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[52],1,[]
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,[120],1,[0.832999]
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,[205],1,[0.432376]
10742,5b9257ee-ee54-4936-a192-12facfa04be7,119 52 160 9 88,0,"[0.829782, 0.689188, 0.495721, 0.384784, 0.153..."


In [77]:
df.categories = df.categories.apply(select_top)
df

In [78]:
df

,id,categories,osd,conf
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[52],1,[]
1,49254842-f712-4875-ad09-57ebf8508bd1,[219],1,"[0.68369, 0.591417, 0.535339, 0.298256]"
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[52],1,[]
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,[160],0,[0.801862]
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,[51],0,[0.561465]
...,...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[52],1,[]
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,[120],1,[0.832999]
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,[205],1,[0.432376]
10742,5b9257ee-ee54-4936-a192-12facfa04be7,[119],0,"[0.829782, 0.689188, 0.495721, 0.384784, 0.153..."


In [79]:
generate_submission_file(df, 'submission_14.csv')

In [13]:
# submission 7
df = pd.read_csv('submission_12_int.csv')
df

,id,categories,osd
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[52],0.9
1,49254842-f712-4875-ad09-57ebf8508bd1,219 52 218 242,0.5
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[52],0.9
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,['160'],0.5
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,['51'],0.5
...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[52],0.9
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,['120'],0.5
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,['205'],0.5
10742,5b9257ee-ee54-4936-a192-12facfa04be7,119 52 160 9 88,0.5


In [73]:
df

,id,categories,osd,conf
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[52],1.0,[]
1,49254842-f712-4875-ad09-57ebf8508bd1,[219],0.5,"[0.68369, 0.591417, 0.535339, 0.298256]"
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[52],1.0,[]
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,[160],0.0,[0.801862]
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,[51],0.0,[0.561465]
...,...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[52],1.0,[]
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,[120],0.5,[0.832999]
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,[205],0.5,[0.432376]
10742,5b9257ee-ee54-4936-a192-12facfa04be7,[119],0.0,"[0.829782, 0.689188, 0.495721, 0.384784, 0.153..."


In [12]:
df.categories = df['categories'].apply(convert_cats)
df

,id,categories,osd
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[ 5 2 ],0.9
1,49254842-f712-4875-ad09-57ebf8508bd1,2 1 9 5 2 2 1 8 2 4 2,0.5
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[ 5 2 ],0.9
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,[ ' 1 6 0 ' ],0.5
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,[ ' 5 1 ' ],0.5
...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[ 5 2 ],0.9
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,[ ' 1 2 0 ' ],0.5
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,[ ' 2 0 5 ' ],0.5
10742,5b9257ee-ee54-4936-a192-12facfa04be7,1 1 9 5 2 1 6 0 9 8 8,0.5


In [ ]:
df['categories'].apply()

In [41]:
df['categories'] = df['categories'].apply(select_top)

In [42]:
df

,id,categories,osd
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[52],1
1,49254842-f712-4875-ad09-57ebf8508bd1,[219],1
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[52],1
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,[160],0
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,[51],0
...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[52],1
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,[120],1
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,[205],1
10742,5b9257ee-ee54-4936-a192-12facfa04be7,[119],0


In [43]:
df[['id', 'categories', 'osd']].to_csv('submission_11.csv', index=False)

In [5]:
df.categories = df.categories.str.split(' ')
df.categories = df.categories.apply(remove_duplicates)
df

,id,categories,osd
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[160],0.9
1,49254842-f712-4875-ad09-57ebf8508bd1,"[219, 52, 218, 242]",0.5
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[160],0.9
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,[160],0.1
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,[51],0.1
...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[160],0.9
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,[120],0.5
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,[205],0.5
10742,5b9257ee-ee54-4936-a192-12facfa04be7,"[119, 52, 160, 9]",0.1


In [13]:
convert_cats(['1', '2']), convert_cats(['1'])

('1 2', '[1]')

In [14]:
df.categories = df.categories.apply(convert_cats)

In [15]:
df

,id,categories,osd
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[160],0.9
1,49254842-f712-4875-ad09-57ebf8508bd1,219 52 218 242,0.5
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[160],0.9
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,[160],0.1
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,[51],0.1
...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[160],0.9
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,[120],0.5
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,[205],0.5
10742,5b9257ee-ee54-4936-a192-12facfa04be7,119 52 160 9,0.1


In [16]:
df[['id', 'categories', 'osd']].to_csv('submission_7.csv', index=False)